In [4]:
import sys
import warnings
import copy
import pandas as pd
import numpy as np
from sklearn import model_selection, preprocessing, ensemble, metrics, linear_model
#, naive_bayes, metrics, svm, decomposition
import xgboost
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE, RandomOverSampler

warnings.simplefilter('ignore')

input_train_tr = f'{sys.path[0]}/Input/ieee-fraud-detection/train_transaction.csv'
input_test_tr = f'{sys.path[0]}/Input/ieee-fraud-detection/test_transaction.csv'
input_train_id = f'{sys.path[0]}/Input/ieee-fraud-detection/train_identity.csv'
input_test_id = f'{sys.path[0]}/Input/ieee-fraud-detection/test_identity.csv'
output_path = f'{sys.path[0]}/output/sample_submission.csv'

output = pd.DataFrame()

train_transaction = pd.read_csv(input_train_tr)
test_transaction = pd.read_csv(input_test_tr)
train_identity = pd.read_csv(input_train_id)
test_identity = pd.read_csv(input_test_id)

#train_ident = train_transaction['TransactionID'].isin(train_identity['TransactionID'])
#test_ident = test_transaction['TransactionID'].isin(test_identity['TransactionID'])

#train_tr = train_transaction[train_ident == False]
tr_dataset = pd.merge(train_transaction, train_identity, how = 'left', on='TransactionID')

#test_tr = test_transaction[test_ident == False]
fin_test_id = pd.merge(test_transaction, test_identity, how = 'left', on='TransactionID')

In [5]:
# label encode the target variable
def encode(dataset):

    encoder = preprocessing.LabelEncoder()

    non_num_cols = [dataset.columns[n] for n,i in enumerate(dataset.dtypes)
                    if i not in ('int64','float64')]

    #Filling missing string values with most common value
    for i in non_num_cols:
        dataset[i+'_mod'] = encoder\
                            .fit_transform(dataset[i].fillna(dataset[i]\
                                                             .replace(np.nan
                                                                      , '0'
                                                                      , regex=True).mode()[0]))

    col_list = [x for x in train_transaction.columns if x not in non_num_cols+['isFraud']]
    
    y = dataset['isFraud']
    
    dataset = dataset[col_list].fillna(0)

    return dataset, y 

# feature engineering
def feature_eng(y, dataset):

    x_train = dataset.fillna(0)
    print(len(x_train),len(y))
    scaler = MinMaxScaler(feature_range = (0,1))

    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    print(len(x_train),len(y))
    #ros = RandomOverSampler(random_state=0)
    #x_train, x_valid = ros.fit_resample(x_train, x_valid)
    x_train, y = SMOTE().fit_resample(x_train, y)
    print(len(x_train),len(y))
    return x_train, y

# algorithm testing
def test_alg(x_train, y_train, x_test, y_test):

    RF = ensemble.RandomForestClassifier(n_jobs=-1) #97.25

    RF.fit(x_train, y_train)
    predictions = RF.predict(x_test)
    
    accuracy = metrics.accuracy_score(predictions, y_test)
    print (f"RF:{accuracy}")
    #return predictions

In [6]:
# mapped id
x_train, y = encode(tr_dataset)
x_train, y = feature_eng(y, x_train)

590540 590540
590540 590540
1139754 1139754


In [13]:
x_train, y_train, x_dev, y_dev = model_selection.train_test_split(x_train, y, test_size = 0.01)

x_train, y_train, x_test, y_test = model_selection.train_test_split(x_train, y_train, test_size = 0.1)

test_alg(x_train, y_train, x_dev, y_dev)
test_alg(x_dev, y_dev, x_test, y_test)
test_alg(x_train, y_train, x_test, y_test)

ValueError: Found input variables with inconsistent numbers of samples: [255830, 265830]